# 014 – Arduino Test (Pump & Trafo)

Quick smoke-test for both Arduinos:
- **PumpLocker** on `COM19` (temp, fan, flow1, flow2)
- **TrafoLocker** on `COM33` (temp, fan)

In [9]:
import sys
import os
import logging
from datetime import datetime
from pathlib import Path

# Make src importable
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))

from devices.arduino.pump_arduino import PumpArduino
from devices.arduino.trafo_arduino import TrafoArduino

In [10]:
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = log_dir / f"014_arduinotest_{timestamp}.log"

logger = logging.getLogger(f"TPG366_PressureTest_{timestamp}")
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(console_handler)

logger.info("Logger initialized.")
print(f"Log file: {log_file}")

2026-02-12 15:26:59,602 - INFO - Logger initialized.


Log file: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\014_arduinotest_20260212_152659.log


In [11]:
# Instantiate both Arduinos
pump  = PumpArduino("pump_locker",  port="COM19", logger=logger)
trafo = TrafoArduino("trafo_locker", port="COM33", logger=logger)

print(pump.get_status())
print(trafo.get_status())

{'device_id': 'pump_locker', 'port': 'COM19', 'baudrate': 9600, 'connected': False, 'timeout': 1.0, 'hk_running': False, 'hk_interval': 1.0, 'external_thread': False}
{'device_id': 'trafo_locker', 'port': 'COM33', 'baudrate': 9600, 'connected': False, 'timeout': 1.0, 'hk_running': False, 'hk_interval': 1.0, 'external_thread': False}


In [12]:
# Connect
pump.connect()
trafo.connect()

2026-02-12 15:27:01,692 - INFO - Connecting to Arduino pump_locker on COM19
2026-02-12 15:27:01,703 - INFO - Connecting to Arduino trafo_locker on COM33


True

In [13]:
# Read a few lines from each Arduino
import time

for i in range(10):
    p = pump.read_arduino_data()
    t = trafo.read_arduino_data()
    if p:
        print(f"Pump:  {p}")
    if t:
        print(f"Trafo: {t}")
    time.sleep(0.5)

Pump:  {'temperature': 19.0, 'fan_power': 40, 'flow_rate_1': 0.59, 'flow_rate_2': 0.43, 'raw_data': '19.00,40,0.59,0.43'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.0, 'fan_power': 40, 'flow_rate_1': 0.57, 'flow_rate_2': 0.44, 'raw_data': '19.00,40,0.57,0.44'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.0, 'fan_power': 40, 'flow_rate_1': 0.57, 'flow_rate_2': 0.44, 'raw_data': '19.00,40,0.57,0.44'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.06, 'fan_power': 40, 'flow_rate_1': 0.57, 'flow_rate_2': 0.44, 'raw_data': '19.06,40,0.57,0.44'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.06, 'fan_power': 40, 'flow_rate_1': 0.57, 'flow_rate_2': 0.44, 'raw_data': '19.06,40,0.57,0.44'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.0, 'fan

2026-02-12 15:27:06,335 - DEBUG - No data received from Arduino.


Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.0, 'fan_power': 40, 'flow_rate_1': 0.6, 'flow_rate_2': 0.42, 'raw_data': '19.00,40,0.60,0.42'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.0, 'fan_power': 40, 'flow_rate_1': 0.6, 'flow_rate_2': 0.42, 'raw_data': '19.00,40,0.60,0.42'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}
Pump:  {'temperature': 19.06, 'fan_power': 40, 'flow_rate_1': 0.6, 'flow_rate_2': 0.46, 'raw_data': '19.06,40,0.60,0.46'}
Trafo: {'temperature': 19.19, 'fan_power': 40, 'raw_data': '19.19,40'}


In [14]:
trafo.start_housekeeping()

2026-02-12 15:27:08,857 - INFO - Using external logger - no additional file logging needed
2026-02-12 15:27:08,858 - INFO - Housekeeping worker started for trafo_locker
2026-02-12 15:27:08,858 - INFO - Housekeeping started (internal mode) - interval: 1.0s


True

2026-02-12 15:27:08,862 - INFO - trafo_locker   COM33   Temp   19.19//degC
2026-02-12 15:27:08,862 - INFO - trafo_locker   COM33   Fan_PWR   40//%


In [15]:
pump.start_housekeeping()

2026-02-12 15:27:09,938 - INFO - Using external logger - no additional file logging needed
2026-02-12 15:27:09,939 - INFO - Housekeeping worker started for pump_locker
2026-02-12 15:27:09,940 - INFO - Housekeeping started (internal mode) - interval: 1.0s


True

2026-02-12 15:27:09,943 - INFO - pump_locker   COM19   Temp   19.06//degC
2026-02-12 15:27:09,943 - INFO - pump_locker   COM19   Fan_PWR   40//%
2026-02-12 15:27:09,944 - INFO - pump_locker   COM19   Flow1   0.6//L/min


In [16]:
trafo.stop_housekeeping()
pump.stop_housekeeping()

2026-02-12 15:27:45,726 - INFO - Housekeeping stopped (internal mode)
2026-02-12 15:27:45,727 - INFO - Housekeeping stopped (internal mode)


True

In [17]:
# Disconnect
pump.disconnect()
trafo.disconnect()

2026-02-12 15:27:46,404 - INFO - Disconnected from Arduino pump_locker
2026-02-12 15:27:46,406 - INFO - Disconnected from Arduino trafo_locker


True